<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [WINE1] - Wine quality prediction with a Dense Network (DNN) using Lightning
  <!-- DESC -->  Another example of regression, with a wine quality prediction!
  <!-- AUTHOR : Achille Mbogol Touye (EFFILIA-MIAI/SIMaP) -->

## Objectives :
 - Predict the **quality of wines**, based on their analysis
 - Understanding the principle and the architecture of a regression with a dense neural network with backup and restore of the trained model. 

The **[Wine Quality datasets](https://archive.ics.uci.edu/ml/datasets/wine+Quality)** are made up of analyses of a large number of wines, with an associated quality (between 0 and 10)  
This dataset is provide by :  
Paulo Cortez, University of Minho, Guimarães, Portugal, http://www3.dsi.uminho.pt/pcortez  
A. Cerdeira, F. Almeida, T. Matos and J. Reis, Viticulture Commission of the Vinho Verde Region(CVRVV), Porto, Portugal, @2009  
This dataset can be retreive at [University of California Irvine (UCI)](https://archive-beta.ics.uci.edu/ml/datasets/wine+quality)


Due to privacy and logistic issues, only physicochemical and sensory variables are available  
There is no data about grape types, wine brand, wine selling price, etc.

- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulphates
- alcohol
- quality (score between 0 and 10)

## What we're going to do :

 - (Retrieve data)
 - (Preparing the data)
 - (Build a model)
 - Train and save the model
 - Restore saved model
 - Evaluate the model
 - Make some predictions


## Step 1 - Import and init


In [1]:
# Import some packages
import os
import sys
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import lightning.pytorch as pl
import torch.nn.functional as F
import torchvision.transforms as T

from IPython.display import Markdown
from importlib import reload
from torch.utils.data import Dataset, DataLoader, random_split
from data_load import WineQualityDataset, Normalize, ToTensor
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger
from torchmetrics.functional.regression import mean_absolute_error, mean_squared_error

import fidle

# Init Fidle environment
run_id, run_dir, datasets_dir = fidle.init('WINE1-Lightning')

<br>**FIDLE - Environment initialization**

Version              : 2.2b7
Run id               : WINE1-Lightning
Run dir              : ./run/WINE1-Lightning
Datasets dir         : /home/achille-touye/fidle-tp/datasets-fidle
Start time           : 01/11/23 13:44:36
Hostname             : achilletouye-Precision-3571 (Linux)
Tensorflow log level : Info + Warning + Error  (=0)
Update keras cache   : False
Update torch cache   : False
Save figs            : ./run/WINE1-Lightning/figs (False)
numpy                : 1.24.4
sklearn              : 1.3.2
yaml                 : 6.0.1
matplotlib           : 3.7.3
pandas               : 2.0.3
torch                : 2.1.0+cu121
torchvision          : 0.16.0+cu121
lightning            : 2.1.0


Verbosity during training : 
- 0 = silent
- 1 = progress bar
- 2 = one line per epoch

In [2]:
fit_verbosity = 1
dataset_name  = 'winequality-red.csv'

Override parameters (batch mode) - Just forget this cell

In [3]:
fidle.override('fit_verbosity', 'dataset_name')

## Step 2 - Retrieve data

In [4]:
csv_file_path=f'{datasets_dir}/WineQuality/origine/{dataset_name}'
datasets=WineQualityDataset(csv_file_path)

display(datasets.data.head(5).style.format("{0:.2f}"))
print('Missing Data : ',datasets.data.isna().sum().sum(), '  Shape is : ', datasets.data.shape)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5.00
1,7.80,0.88,0.00,2.60,0.10,25.00,67.00,1.00,3.20,0.68,9.80,5.00
2,7.80,0.76,0.04,2.30,0.09,15.00,54.00,1.00,3.26,0.65,9.80,5.00
3,11.20,0.28,0.56,1.90,0.07,17.00,60.00,1.00,3.16,0.58,9.80,6.00
4,7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5.00


Missing Data :  0   Shape is :  (1599, 12)


## Step 3 - Preparing the data


### 3.1 - Data normalization
**Note :** 
 - All input features must be normalized.  
 - To do this we will subtract the mean and divide by the standard deviation for each input features. 
 - Then we convert numpy array features and target **(quality)** to torch tensor   

In [5]:
transforms=T.Compose([Normalize(csv_file_path), ToTensor()])

dataset=WineQualityDataset(csv_file_path,transform=transforms)

In [6]:
display(Markdown("before normalization :"))
display(datasets[:]["features"])

print()

display(Markdown("After normalization :"))
display(dataset[:]["features"])

before normalization :

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
       [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]],
      dtype=float32)

After normalization :

tensor([[-0.5282,  0.9616, -1.3910,  ...,  1.2882, -0.5790, -0.9599],
        [-0.2985,  1.9668, -1.3910,  ..., -0.7197,  0.1289, -0.5846],
        [-0.2985,  1.2967, -1.1857,  ..., -0.3311, -0.0481, -0.5846],
        ...,
        [-1.1600, -0.0995, -0.7237,  ...,  0.7053,  0.5419,  0.5415],
        [-1.3897,  0.6544, -0.7750,  ...,  1.6769,  0.3059, -0.2092],
        [-1.3323, -1.2165,  1.0217,  ...,  0.5110,  0.0109,  0.5415]])

### 3.2 - Split data
We will use 80% of the data for training and 20% for validation.  
x will be the features data of the analysis and y the target (quality)

In [7]:
# ---- Split => train, test
#
data_train_len = int(len(dataset)*0.8)            # get 80 %
data_test_len  = len(dataset) -data_train_len     # test = all - train

# ---- Split => x,y with random_split
#
data_train_subset, data_test_subset=random_split(dataset, [data_train_len, data_test_len])       

                                                
x_train = data_train_subset[:]["features"]
y_train = data_train_subset[:]["quality" ]

x_test  = data_test_subset [:]["features"]
y_test  = data_test_subset [:]["quality" ]


print('Original data shape was : ',dataset.data.shape)
print('x_train : ',x_train.shape, 'y_train : ',y_train.shape)
print('x_test  : ',x_test.shape,  'y_test  : ',y_test.shape)

Original data shape was :  (1599, 12)
x_train :  torch.Size([1279, 11]) y_train :  torch.Size([1279, 1])
x_test  :  torch.Size([320, 11]) y_test  :  torch.Size([320, 1])


### 3.3 -  For Training model use Dataloader
The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in minibatches, reshuffle the data at every epoch to reduce model overfitting. DataLoader is an iterable that abstracts this complexity for us in an easy API.

In [8]:
# train bacth data
train_loader= DataLoader(
  dataset=data_train_subset, 
  shuffle=True, 
  batch_size=20,
  num_workers=2  
)


# test bacth data
test_loader= DataLoader(
  dataset=data_test_subset, 
  shuffle=False, 
  batch_size=20,
  num_workers=2
)

## Step 4 - Build a model
More informations about : 
 - [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
 - [Activation](https://www.tensorflow.org/api_docs/python/tf/keras/activations)
 - [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
 - [Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [9]:
class LitRegression(pl.LightningModule):
    
    def __init__(self,in_features=11):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features, 128),                               # hidden layer 1
            nn.ReLU(),                                                 # activation function 
            nn.Linear(128, 128),                                       # hidden layer 2
            nn.ReLU(),                                                 # activation function
            nn.Linear(128, 1))                                         # output layer   
    
    def forward(self, x):                                              # forward pass
        x = self.model(x)
        return x        
        
     # optimizer
    def configure_optimizers(self):                              
        optimizer = torch.optim.RMSprop(self.parameters(),lr=1e-4)
        return optimizer 
        
    
    def training_step(self, batch, batch_idx):
        # defines the train loop.
        x_features, y_target = batch["features"],batch["quality"]
        
        # forward pass
        y_pred = self.model(x_features)

        # loss function MSE
        loss   = F.mse_loss(y_pred, y_target)                           

        # metrics mae
        mae    = mean_absolute_error(y_pred,y_target) 

        # metrics mse
        mse    = mean_squared_error(y_pred,y_target)                    
        
        metrics= {"train_loss": loss, 
                   "train_mae" : mae, 
                   "train_mse" : mse
                  }
        
        # logs metrics for each training_step
        self.log_dict(metrics, 
                      on_step  = False,                     
                      on_epoch = True, 
                      logger   = True,
                      prog_bar = True,     
                     )
        return loss      

        
    def validation_step(self, batch, batch_idx):
        
        # defines the val loop.
        x_features, y_target = batch["features"],batch["quality"]

        # forward pass
        y_pred = self.model(x_features)

        # loss function MSE
        loss   = F.mse_loss(y_pred, y_target)                             

        # metrics
        mae    = mean_absolute_error(y_pred,y_target)

        # metrics
        mse    = mean_squared_error(y_pred,y_target)                          

        
        metrics= {"val_loss": loss, 
                   "val_mae" : mae, 
                   "val_mse" : mse
                  }
       
        # logs metrics for each validation_step 
        self.log_dict(metrics,                               
                      on_step  = False,                     
                      on_epoch = True, 
                      logger   = True,
                      prog_bar = True,     
                     )

        return metrics
            
   

## 5 - Train the model
### 5.1 - Get it

In [10]:
reg=LitRegression(in_features=11)
print(reg) 

LitRegression(
  (model): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


### 5.2 - Add callback

In [11]:
os.makedirs('./run/models', exist_ok=True)
save_dir = "./run/models/"
filename ='best-model-{epoch}-{val_loss:.2f}'

savemodel_callback = pl.callbacks.ModelCheckpoint(dirpath=save_dir, 
                                                  filename=filename,
                                                  save_top_k=1, 
                                                  verbose=False, 
                                                  monitor="val_loss"
                                                 )

### 5.3 - Train it

In [12]:
# loggers data
logger  = TensorBoardLogger(save_dir='Wine_logs',name="reg_logs")

In [13]:
# train model
trainer = pl.Trainer(accelerator='auto',
                     max_epochs=100,
                     logger=logger,
                     callbacks=[savemodel_callback])

trainer.fit(model=reg, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-11-01 13:44:37.819090: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-01 13:44:37.820282: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 13:44:37.844348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 13:44:38.255107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

Sanity Checking: |                                                                                         | 0…

Training: |                                                                                                | 0…

Validation: |                                                                                              | 0…

Validation: |                                                                                              | 0…

...

Validation: |                                                                                              | 0…

Validation: |                                                                                              | 0…

Validation: |                                                                                              | 0…

Validation: |                                                                                              | 0…

`Trainer.fit` stopped: `max_epochs=100` reached.


## Step 6 - Evaluate
### 6.1 - Model evaluation
MAE =  Mean Absolute Error (between the labels and predictions)  
A mae equal to 3 represents an average error in prediction of $3k.

In [14]:
score=trainer.validate(model=reg, dataloaders=test_loader, verbose=False)

print('x_test / loss      : {:5.4f}'.format(score[0]['val_loss']))
print('x_test / mae       : {:5.4f}'.format(score[0]['val_mae']))
print('x_test / mse       : {:5.4f}'.format(score[0]['val_mse']))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |                                                                                              | 0…

x_test / loss      : 0.4608
x_test / mae       : 0.5277
x_test / mse       : 0.4608


### 6.2 - Training history

In [15]:
# launch Tensorboard 
%reload_ext tensorboard
%tensorboard --logdir=Wine_logs/reg_logs/

## Step 7 - Restore a model :

### 7.1 - Reload model

In [16]:
# Load the model from a checkpoint
loaded_model = LitRegression.load_from_checkpoint(savemodel_callback.best_model_path)
print("Loaded:")
print(loaded_model)

Loaded:
LitRegression(
  (model): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


### 7.2 - Evaluate it :

In [17]:
score=trainer.validate(model=loaded_model, dataloaders=test_loader, verbose=False)

print('x_test / loss      : {:5.4f}'.format(score[0]['val_loss']))
print('x_test / mae       : {:5.4f}'.format(score[0]['val_mae']))
print('x_test / mse       : {:5.4f}'.format(score[0]['val_mse']))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |                                                                                              | 0…

x_test / loss      : 0.4533
x_test / mae       : 0.5209
x_test / mse       : 0.4533


### 7.3 - Make a prediction

In [18]:
# ---- Pick n entries from our test set
n = 200
ii = np.random.randint(1,len(x_test),n)
x_sample = x_test[ii]
y_sample = y_test[ii]

In [19]:
# ---- Make a predictions :

# Sets the model in evaluation mode.
loaded_model.eval() 

# Perform inference using the loaded model
y_pred = loaded_model( x_sample )

In [20]:
# ---- Show it
print('Wine    Prediction   Real   Delta')
for i in range(n):
    pred   = y_pred[i][0].item()
    real   = y_sample[i][0].item()
    delta  = real-pred
    print(f'{i:03d}        {pred:.2f}       {real}      {delta:+.2f} ')

Wine    Prediction   Real   Delta
000        5.81       6.0      +0.19 
001        4.93       5.0      +0.07 
002        6.22       7.0      +0.78 
003        5.21       3.0      -2.21 
004        6.46       7.0      +0.54 
005        5.03       5.0      -0.03 
006        6.59       7.0      +0.41 
007        5.86       5.0      -0.86 
008        6.55       7.0      +0.45 
009        6.71       7.0      +0.29 
010        5.65       5.0      -0.65 
011        5.25       5.0      -0.25 
012        6.31       6.0      -0.31 
013        6.94       8.0      +1.06 
014        5.55       5.0      -0.55 
015        5.47       7.0      +1.53 
016        5.49       5.0      -0.49 
017        6.81       8.0      +1.19 
018        5.40       5.0      -0.40 
019        5.86       6.0      +0.14 
020        5.15       6.0      +0.85 
021        4.76       5.0      +0.24 
022        5.34       6.0      +0.66 
023        5.55       7.0      +1.45 
024        6.02       7.0      +0.98 
025        5.92 

In [21]:
fidle.end()

**End time :** 01/11/23 13:46:01  
**Duration :** 00:01:25 977ms  
This notebook ends here :-)  
[https://fidle.cnrs.fr](https://fidle.cnrs.fr)

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>